# Primeiros passos com MicroPython e ESP8266

Por Fábio Souza https://about.me/fabio.souza

[Acesse os de introdução SLIDES](https://docs.google.com/presentation/d/1G-glUcrQwYLMisaA1JuDkSy9KEdgCUqQ/edit#slide=id.p26)



## Instalação do Firmware no ESP8266


- Acesse a página de Download do [Firmware do MicroPython](https://micropython.org/download#esp8266)
- Baixe a ultima versão, no meu caso foi: [esp8266-20190529-v1.11.bin](https://micropython.org/resources/firmware/esp8266-20190529-v1.11.bin)
- Intruções para gravar o Firmware: [Deploying the firmware](http://docs.micropython.org/en/latest/esp8266/tutorial/intro.html#deploying-the-firmware)


### Linux

Instalar o epstool: `pip install esptool`

conectar a placa no computador através da USB

Verificar em qual porta está conectada: `ls /dev/tty*` 

Apagar o que estiver carregado na placa: ` esptool.py --port /dev/ttyUSB0 erase_flash`

Gravar o Firmware do microPython na flash: `esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect 0 esp8266-20190529-v1.11.bin`

Abra um terminal serial, por exemplo o Picocom: `picocom --baud 115200 /dev/ttyUSB0`

digite: `print("Hello world!")`


Pronto a sua placa já está com o Micropython!



### Windows

[Primeiros passos com MicroPython no ESP8266](https://www.embarcados.com.br/micropython-no-esp8266)



# Primeiros passos com MicroPython no ESP8266 usando o Jupyter

## Comunicação serial - Configura e conecta

- baudrate 115200 bps
- porta tty*

In [3]:
%serialconnect to --port=/dev/ttyUSB1 --baud=115200

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


### Hello World

In [2]:
print ('hello esp8266')

hello esp8266


### GPIO

In [3]:
from machine import Pin

### Digital Output

In [4]:
led = Pin(2, Pin.OUT)

In [5]:
led.value(1)

In [6]:
led.value(0)

In [7]:
led.value(1)

In [8]:
led.off()

In [9]:
led.on()

### Blinkiing the LED

In [10]:
from time import sleep

In [11]:
for i in range(10):
    led.off()
    sleep(0.2)
    led.on()
    sleep(0.2)

### Buzzer

In [14]:
buzzer = Pin(15, Pin.OUT)
buzzer.value(1)
sleep(0.2)
buzzer.value(0)


### Digital Input

In [15]:
button = Pin(13, Pin.IN, Pin.PULL_UP)

# Read button state:
print(button.value())

1


In [16]:
print (button.value())
if button.value() == 0:
    led.on()
else:
    led.off()

1


## PWM (pulse Width Modulation)

![PWM](https://exploreembedded.com/wiki/images/5/54/0_LPC1768_PWM.gif){:class="img-responsive"}


In [17]:
from machine import Pin, PWM

In [18]:
pwm0 = PWM(Pin(2)) 

In [19]:
print (pwm0.freq())

500


In [20]:
pwm0.freq(1000)

In [21]:
print (pwm0.duty())

0


In [22]:
pwm0.duty(600)

In [23]:
pwm0.deinit() 

In [24]:
pwm2 = PWM(Pin(2), freq=500, duty=900)

In [25]:
from machine import Pin, PWM
pwm0 = PWM(Pin(2), freq=1000, duty=0)
for i in range (0,1023,20):
    pwm0.duty(i)
    sleep(0.1)
    pwm0.duty(0)

.

In [26]:
pwm0.deinit()

In [29]:
from machine import Pin, PWM
import time
tempo = 2
tones = {
    'c': 262,
    'd': 294,
    'e': 330,
    'f': 349,
    'g': 392,
    'a': 440,
    'b': 494,
    'C': 523,
    ' ': 0,
}
beeper = PWM(Pin(15, Pin.OUT), freq=440, duty=512)
melody = 'cdefgabC'
rhythm = [16, 16, 16, 16, 16, 16, 16, 16]

for tone, length in zip(melody, rhythm):
    beeper.freq(tones[tone])
    time.sleep(tempo/length)
beeper.deinit()

## Conversor Analógico para Digital

In [32]:
from machine import ADC

In [33]:
adc0 = ADC(0)

In [35]:
print("adc0=",adc0.read())    #read value, 0-1024

adc0= 624


In [36]:
for i in range(10):
    print("adc0=",adc0.read())    #read value, 0-1024
    sleep(0.5)

adc0= 446
adc0= 448
adc0= 448
adc0= 447
adc0= 446
adc0= 637
adc0= 629
adc0= 716
adc0= 675
adc0= 450


## Comunicação I2C

In [37]:
from machine import I2C

In [38]:
i2c = I2C(scl=Pin(5), sda=Pin(4), freq=100000)

In [39]:
print (i2c.scan())

[56, 60]


In [40]:
i2c.writeto(0x38, bytes(0xFF))                  #Start PCF8574

In [45]:
buf = bytearray(1)
buf[0] = 255
i2c.writeto(0x38,buf)
buf[0] = 0xff &~(1<<0|1<<1|1<<2|1<<3)
i2c.writeto(0x38,buf)
pos_led = 0
buf[0] = 0xF0|(1<<pos_led)
i2c.writeto(0x38,buf)

In [50]:
buf = i2c.readfrom(0x38,1) 
z = buf[0]
z=z>>4
print("%#5.2X"% (z))


 0X0B


In [51]:
if z == 0x07:
    print('Tecla:1')
elif z == 0x0B:
    print('Tecla:2')
elif z == 0X0D:
    print('Tecla:3')
elif z == 0x0E:
    print('Tecla:4')

Tecla:2


## Display OLED 128x64 0.96" I2C

![OLED](https://uploads.filipeflop.com/2017/07/5LE19.png)

In [52]:
import ssd1306

In [53]:
i2c = I2C(scl=Pin(5), sda=Pin(4), freq=100000)  #Init i2c

In [54]:
oled =ssd1306.SSD1306_I2C(128,64,i2c,0x3c)

In [55]:
oled.fill(0)
oled.show()

In [56]:
oled.text("Hello World",5,5)
oled.show()

In [57]:
oled.invert(True)  #True or False

In [58]:
oled.invert(False)

In [59]:
oled.rect(0, 0, 128, 20, 1)
oled.show()

In [60]:
ICON = [
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 1, 1, 0, 0, 0, 1, 1, 0],
    [ 1, 1, 1, 1, 0, 1, 1, 1, 1],
    [ 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [ 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [ 0, 1, 1, 1, 1, 1, 1, 1, 0],
    [ 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [ 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [ 0, 0, 0, 0, 1, 0, 0, 0, 0],
]

for y, row in enumerate(ICON):
    for x, c in enumerate(row):
        oled.pixel(x+100, y+5, c)

oled.show()

In [61]:
oled.fill(0)
oled.show()

## DHT11

O DHT11 é um sensor de temperatura e umidade que permite fazer leituras de temperaturas entre 0 a 50 Celsius e umidade entre 20 a 90%.

![DHT11](https://uploads.filipeflop.com/2017/07/Dht11.jpg)


In [62]:
from dht import DHT11

In [63]:
d = DHT11(Pin(2))

In [64]:
d.measure()

In [65]:
temp = '{:.1f}'.format(d.temperature())
print (temp)

25.0


In [66]:
hmdty = '{:.1f}'.format(d.humidity() )
print (hmdty)

18.0


In [67]:
oled.fill(0) 
oled.text('T:', 0, 20)
oled.text(temp ,20,20)                         
oled.text('H:', 0, 40)
oled.text(hmdty ,20,40)                       
oled.show()        

## LED Neopixel

São LEDs RGBs endereçaveis que podem ser controlados individualmente com apenas um pinos do microcontrolador.

![Neopixel](https://i.pinimg.com/originals/4b/d3/f3/4bd3f3a46574eebc0f07c7b6067dcec4.gif)

In [68]:
from neopixel import NeoPixel

In [69]:
pin = Pin(0, Pin.OUT)                           # set GPIO0 to output to drive NeoPixels
np = NeoPixel(pin, 6)                           # create NeoPixel driver on GPIO0 for 6 pixels

In [74]:
np[1] = (100,128, 50)     #Set LED color         
np.write()              # write data to all pixels

In [77]:
for x in range(0, 6):
    np[x] = (0,0, 255)      # set pixel color
    np.write()              # write data to all pixels
    sleep(0.2)              #delay

In [78]:
for x in range(5, -1,-1):
    np[x] = (0,0, 0)        # apaga pixels
    np.write()              # escreve valor nos leds
    sleep(0.2)              #delay    

.

# Programando a placa

É necessário o [Ampy -  Adafruit MicroPython Tool](https://github.com/pycampers/ampy)


Instalação: `sudo pip3 install adafruit-ampy`

Listando arquivos: `ampy --port /dev/ttyUSB0 ls`

Variável de Ambiente para porta: `export AMPY_PORT=/dev/ttyUSB0 `

Listar arquivos sem passar porta: `ampy ls`

Ler arquivo na placa: `ampy get boot.py`

Enviar arquivo para a placa: `ampy put main.py`

Remover arquivo da placa: `ampy rm main.py`

Executar arquivo: `ampy run blink.py`


help `ampy --help`

# Referências

- [MicroPython documentation](http://docs.micropython.org/en/latest/index.html)
- [Firmware Download](https://micropython.org/download#esp8266)
- [Quick reference for the ESP8266](http://docs.micropython.org/en/latest/esp8266/quickref.html)
- [Welcome to Micropython on ESP8266 Workshop’s documentation!](https://micropython-on-esp8266-workshop.readthedocs.io/en/latest/index.html)
- [Welcome to Micropython on ESP8266 Workshop’s documentation!](https://micropython-on-wemos-d1-mini.readthedocs.io/en/latest/index.html)
- [MicroPython Basics: Load Files & Run Code](https://cdn-learn.adafruit.com/downloads/pdf/micropython-basics-load-files-and-run-code.pdf)
- [MicroPython Sublime Text Plugin](https://github.com/gepd/uPiotMicroPythonTool)
- [Micropython IDE for VSCode](https://marketplace.visualstudio.com/items?itemName=dphans.micropython-ide-vscode)
- [Jupyter MicroPython Kernel](https://github.com/goatchurchprime/jupyter_micropython_kernel)




### Artigos que escrevi:

Primeiros passos com MicroPython no ESP8266
- [Primeiros passos com MicroPython no ESP8266](https://www.embarcados.com.br/micropython-no-esp8266/)
- [Placa de testes para MicroPython com ESP8266](https://www.embarcados.com.br/placa-de-testes-para-micropython-com-esp8266/)
- [Entradas e Saídas digitais no MicroPython com ESP8266](https://www.embarcados.com.br/entradas-e-saidas-digitais-micropython-esp8266/)
- [Entrada analógica com MicroPython no ESP8266](https://www.embarcados.com.br/entrada-analogica-com-micropython-no-esp8266/)

Programe a BBC micro:bit com MicroPython - Hello World!
- [Programe a BBC micro:bit com MicroPython - Hello World!](https://www.embarcados.com.br/bbc-microbit-com-micropython-hello-world/)
- [Programe a micro:bit com o Mu](https://www.embarcados.com.br/programe-a-microbit-com-o-mu/)
- [Enviando comandos em tempo real para a micro:bit com o REPL](https://www.embarcados.com.br/comandos-para-a-microbit-com-o-repl/)
- [Acionando a matriz de LEDs da micro:bit com MicroPython](https://www.embarcados.com.br/matriz-de-leds-da-microbit-com-micropython/)
- [Leitura dos botões da Micro:bit com MicroPython](https://www.embarcados.com.br/leitura-dos-botoes-da-microbit-com-micropython/)

